In [1]:
%load_ext autoreload
%autoreload 2

In [56]:
import sys
sys.path.append("./")

### Q1. Install the package

In [1]:
!mlflow --version

mlflow, version 2.3.2


### Q2. Download and preprocess the data

In [55]:
!pwd

/Users/rguarise/Documents/mlops-zoomcamp/homeworks/02-experiment-tracking


In [58]:
!python ./utils/preprocess_data.py --raw_data_path $HOME/Documents/mlops-zoomcamp/data/ --dest_path ./output

In [59]:
!ls -l ./output

total 17320
-rw-r--r--  1 rguarise  1010544492   153660 Jun  1 21:41 dv.pkl
-rw-r--r--  1 rguarise  1010544492  2632817 Jun  1 21:41 test.pkl
-rw-r--r--  1 rguarise  1010544492  2146163 Jun  1 21:41 train.pkl
-rw-r--r--  1 rguarise  1010544492  2336393 Jun  1 21:41 val.pkl


### Q3. Train a model with autolog

In [70]:
import mlflow
import utils.train as train
mlflow.set_tracking_uri("http://127.0.0.1:5003")
mlflow.set_experiment("random-forest-first-run")

2023/06/01 21:47:57 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-first-run' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/5', creation_time=1685666877999, experiment_id='5', last_update_time=1685666877999, lifecycle_stage='active', name='random-forest-first-run', tags={}>

In [71]:
mlflow.sklearn.autolog()

In [68]:
with mlflow.start_run():
    train.run_train("/Users/rguarise/Documents/mlops-zoomcamp/homeworks/02-experiment-tracking/output/")

RandomForestRegressor(max_depth=10, random_state=0)


### Q4. Tune model hyperparameters

In [9]:
mlflow.sklearn.autolog(disable=True)

In [20]:
import utils.hpo as hpo
with mlflow.start_run():
    hpo.run_optimization("/Users/rguarise/Documents/mlops-zoomcamp/homeworks/02-experiment-tracking/output/",1)

[I 2023-06-01 20:20:36,913] A new study created in memory with name: no-name-f8fe2310-9822-4e23-a405-ee1510b1ee11
[I 2023-06-01 20:20:38,501] Trial 0 finished with value: 2.451379690825458 and parameters: {'n_estimators': 25, 'max_depth': 20, 'min_samples_split': 8, 'min_samples_leaf': 3}. Best is trial 0 with value: 2.451379690825458.


### Q5. Promote the best model to the model registry

In [83]:
import utils.register_model as register_model

register_model.run_register_model("/Users/rguarise/Documents/mlops-zoomcamp/homeworks/02-experiment-tracking/output/",4)

Registered model 'best_model' already exists. Creating a new version of this model...
2023/06/01 21:56:45 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: best_model, version 2


test_rmse: 2.294745868566162


Created version '2' of model 'best_model'.
